### はじめに

<div align="center">
    <img alt="Logo" src="https://hikettei.github.io/cl-waffe-docs/cl-waffe.png" width="45%">
</div>

cl-waffe2はANSI Common Lisp上に、深層学習モデルの設計や学習に特化した行列演算ライブラリとそれに関連するパッケージを提供するフレームワークです。このNotebookでは複数回に分けてcl-waffe2の基本的な使い方をサンプルコードと共に提供します。

cl-waffe2のインストール方法やドキュメントについては、[公式ドキュメント](https://hikettei.github.io/cl-waffe2/)をご活用ください。


### common-lisp-jupyter

Jupyter Notebook及びJupyter Labでこのファイルをご覧になっている場合、付属しているセルを実行するためには、[common-lisp-jupyter](https://github.com/yitzchak/common-lisp-jupyter)という拡張ライブラリを予め導入していただく必要があります。（ライブラリのReadme.mdファイル及び公式ドキュメントをご覧ください）

### cl-waffe2を読み込む

cl-waffe2は執筆時点(2023/08/27)では開発途中で、まだQuicklispに登録がされていません。そのため動作するには[オリジナルのGithubリポジトリ](https://github.com/hikettei/cl-waffe2.git)からリポジトリをクローンしていただいたのち、`cl-waffe2.asd`ファイルを読み込んでいただく必要があります。

In [1]:
(load "../../../cl-waffe2.asd") ;; 相対パスが読み込めない場合適度調節してください

(asdf:load-system :cl-waffe2 :silent t)

T

T

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining PERFORM (#<STANDARD-CLASS ASDF/LISP-ACTION:TEST-OP> #<SB-MOP:EQL-SPECIALIZER #<SYSTEM "cl-waffe2/test">>) in DEFMETHOD
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining LPARALLEL.KERNEL:KERNEL-NAME in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFGENERIC: redefining CL-WAFFE2/VM.GENERIC-TENSOR:CURRENT-BACKEND-STATE in DEFGENERIC
SB-KERNEL:REDEFINITION-WITH-DEFGENERIC: redefining CL-WAFFE2/VM.NODES:FORWARD in DEFGENERIC
SB-KERNEL:REDEFINITION-WITH-DEFGENERIC: redefining CL-WAFFE2/VM.NODES:BACKWARD in DEFGENERIC
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE2/VM.GENERIC-TENSOR:MOVETENSOR-P in DEFUN


## パッケージ管理

まず`section-0-basic`パッケージを定義し、そこでAPIを試すことにしましょう。

cl-waffe2は提供する機能ごとにパッケージの名前空間を分離させています。一部ですが：

### 中心システム

| package | description |
| ------- | ----------- |
| :cl-waffe2/vm | cl-waffe2が動作する仮想マシンです。cl-waffe2 IRに関するAPIなど |
| :cl-waffe2/vm.nodes| AbstractNodeに関する拡張的な機能を提供します。 |
| :cl-waffe2/vm.generic-tensor | AbstractTensorに関する拡張的な機能を提供します。 |

### API

| package | description |
| ------- | ----------- |
| :cl-waffe2 | ネットワーク定義や設定に関するユーティリティを提供します |
| :cl-waffe2/base-impl | !addや!reshapeなど基本的なAPIを提供します |
| :cl-waffe2/distributions | randn関数など、テンソルを確率分布から初期化するためのAPIを提供しています |
| :cl-waffe2/nn | 回帰モデルやCNN, 誤差関数など基本的なネットワークを提供します |
| :cl-waffe2/optimizers | AdamやSGDなど最適化関数の実装を提供します |

### バックエンドの一覧

| package | description |
| ------- | ----------- |
| :cl-waffe2/backends.lisp | ANSI Common Lisp上で実装されたバックエンドです。 |
| :cl-waffe2/backends.cpu | OpenBLASやSIMD Extensionなど、外部ライブラリの力を借りて高速化するバックエンドです。|
| :cl-waffe2/backends.jit.cpu | (試験的) cl-waffe2のコードをCにJITコンパイルすることで動作するバックエンドです。 |
| :cl-waffe2/backends.jit.lisp | (廃止予定) cl-waffe2のコードをLispにJITコンパイルすることで動作するバックエンドです。 |

In [3]:
(defpackage :section-0-basic
    (:use
     :cl
     :cl-waffe2
     :cl-waffe2/base-impl
     :cl-waffe2/vm
     :cl-waffe2/vm.nodes
     :cl-waffe2/vm.generic-tensor
     :cl-waffe2/distributions))

(in-package :section-0-basic)

#<PACKAGE "SECTION-0-BASIC">

#<PACKAGE "SECTION-0-BASIC">

## 基本的なデータ型

数学では, 一つの数値をスカラー, 一次元の行列をVector, 二次元の行列をMatrixのように表現します。

$$ 1 $$

$$ (a_1,a_2,\dots,a_n) $$

\begin{pmatrix}
1 & 2 \\
3 & 4 \\
\end{pmatrix}

便宜上、cl-waffe2ではこれら全てのデータ型を内包する`AbstractTensor`というデータ型を用いて計算を進めていきます。`:cl-waffe2/vm.generic-tensor`パッケージが提供する[make-tensor](https://hikettei.github.io/cl-waffe2/generic-tensor/#function-make-tensor)関数を用いることで新しいTensorを作成することができます。

更に、[:cl-waffe2/distributions](https://hikettei.github.io/cl-waffe2/distributions/)パッケージは確率分布からテンソルをサンプリングする様々な関数を提供しています。

In [5]:
(make-tensor 1)

{SCALARTENSOR[float]   
    1.0
  :facet :exist
  :requires-grad NIL
  :backward NIL}

In [7]:
(make-tensor '(10 10) :initial-element 1.0)

{CPUTENSOR[float] :shape (10 10)  
  ((1.0 1.0 1.0 ~ 1.0 1.0 1.0)           
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0)   
        ...
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0)
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

In [8]:
(randn `(10 10)) ;; ガウス分布からサンプリング

{CPUTENSOR[float] :shape (10 10)  
  ((0.1371898    0.98653716   -0.41064423  ~ -2.1985002   0.64959      0.37606597)                    
   (-0.990578    1.8650554    1.549668     ~ 2.007281     0.2079959    1.2413626)   
                 ...
   (-0.14403512  -0.13514197  -0.2724243   ~ 0.59860396   0.87737995   -0.39738473)
   (0.3234897    1.256502     -0.66197085  ~ 1.356871     0.502143     0.6284781))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

## AbstractTensor

<div align="center">
    <img alt="Logo" src="../assets/AbstractTensor.png" width="45%">
</div>

`storage`に相当するデータ型(例: `fixnum`や`Common Lisp標準配列`など...)を`AbstractTensor`でラップすることで、更に以下の情報を付与することができます。

- `:requires-grad` 逆伝播をしたのちに、勾配を求めるかどうか

- `:backward` 計算ノードの情報

- `:facet` テンソルのメモリ割り当ての状態 `:exist`であれば存在する `:input`であれば未割り当て

etc...

また、`AbstractTensor`と`storage`のデータ型は`change-facet`関数を用いて強制的に行うことができます。また、[convert-tensor-facet](https://hikettei.github.io/cl-waffe2/utils/#generic-convert-tensor-facet)メソッドを追加することで任意の組み合わせの変換を拡張することが可能です。

```lisp
;; 使い方
(change-facet 対象 :direction 変換先のデータ型)
```

In [15]:
(change-facet #2A((1 2 3)
		          (4 5 6)
		          (7 8 9))
	       :direction 'AbstractTensor)

{CPUTENSOR[int32] :shape (3 3)  
  ((1 2 3)
   (4 5 6)
   (7 8 9))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

In [10]:
(change-facet (ax+b `(3 3) 0 1) :direction 'array)

#2A((1.0 1.0 1.0) (1.0 1.0 1.0) (1.0 1.0 1.0))

In [14]:
;; change-facet前後でテンソルのポインタは共有されています
;; そのため、AbstractTensorを一旦CL配列にして編集した後、AbstractTensorに戻すといった挙動も可能です。
;; with-facetマクロはchange-facet関数を呼び出した後、自動でa*に結果をbindingしてくれます。

;; 1で埋め尽くされた3x3行列の対角を0.0で埋める
(let ((a (ax+b `(3 3) 0 1)))
  (with-facet (a* (a :direction 'array))
    (setf (aref a* 0 0) 0.0)
    (setf (aref a* 1 1) 0.0)
    (setf (aref a* 2 2) 0.0))
   a)

{CPUTENSOR[float] :shape (3 3)  
  ((0.0 1.0 1.0)
   (1.0 0.0 1.0)
   (1.0 1.0 0.0))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

## 演算をする

;; [TODO] 書くこと: 遅延評価 逆伝播 with-devices disassemble-waffe2-ir change-facet